In [1]:
%run 10_ngram_common.ipynb

/work/van-speech-nlp/jindaznb/asrenv/lib/python3.10/site-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(


In [22]:
# Read the dataset
data_df = pd.read_csv('torgo.csv')
dataset_csv = load_dataset('csv', data_files='torgo.csv')

speakers = data_df['speaker_id'].unique()

print(f'Speakers: {", ".join(speakers)}')

Speakers: F01, F03, F04, FC01, FC02, FC03, M01, M02, M03, M04, M05, MC01, MC02, MC03, MC04


In [27]:
dataset_csv

DatasetDict({
    train: Dataset({
        features: ['session', 'audio', 'text', 'speaker_id'],
        num_rows: 16394
    })
})

In [37]:
atypical_speaker_texts = {}

for speaker_id in atypical_speakers:
    atypical_speaker_data = dataset_csv['train'].filter(lambda x: x['speaker_id'] == speaker_id)
    texts = atypical_speaker_data['text']
    
    cleaned_texts = []
    for text in texts:
        cleaned_text = ' '.join(re.sub(r'[^a-zA-Z0-9]', '', word.lower()) for word in text.split())
        cleaned_texts.append(cleaned_text)
        
    atypical_speaker_texts[speaker_id] = cleaned_texts


for speaker_id, texts in atypical_speaker_texts.items():
    print(f"{speaker_id}: {len(texts)}")

F01: 228
F03: 1075
F04: 667
M01: 739
M02: 766
M03: 800
M04: 652
M05: 573


In [38]:
os.makedirs("atypical_speaker_texts", exist_ok=True)

for speaker_id, texts in atypical_speaker_texts.items():
    with open(f"atypical_speaker_texts/{speaker_id}_texts.txt", "w") as file:
        for text in texts:
            file.write(text + "\n")

In [47]:
arpa_dir = "arpa_files"
os.makedirs(arpa_dir, exist_ok=True)

for file_name in os.listdir("atypical_speaker_texts"):
    if file_name.endswith(".txt"):
        speaker_id = file_name.split("_")[0]
        arpa_file = f"{arpa_dir}/{speaker_id}_3gram.arpa"
        txt_file = f"atypical_speaker_texts/{file_name}"
        !kenlm/build/bin/lmplz -o 3 < "{txt_file}" > "{arpa_file}" -S 10% --discount_fallback

=== 1/5 Counting and sorting n-grams ===
Reading /work/van-speech-nlp/jindaznb/jslpnb/torgo_error_correction/atypical_speaker_texts/F01_texts.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 556 types 193
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:2316 2:4670902784 3:8757943296
Substituting fallback discounts for order 1: D1=0.5 D2=1 D3+=1.5
Substituting fallback discounts for order 2: D1=0.5 D2=1 D3+=1.5
Statistics:
1 193 D1=0.857868 D2=1.63234 D3+=1.28426
2 321 D1=0.5 D2=1 D3+=1.5
3 231 D1=0.5 D2=1 D3+=1.5
Memory estimate for binary LM:
type        B
probing 16876 assuming -p 1.5
probing 19576 assuming -r models -p 1.5
trie     8999 without quantization
trie     9520 assuming -q 8 -b 8 quantization 
trie     8957 assuming -a 22 array pointer compression
trie     9479 assuming -a 22 -q 8 -b 8

In [51]:
atypical_speakers = ['F01', 'F03', 'F04', 'M01', 'M02', 'M03', 'M04', 'M05']

for speaker in atypical_speakers:
    arpa_file = f"torgo_arpa_files/{speaker}_3gram.arpa"
    output_file = f"torgo_arpa_files/{speaker}_unigram.txt"

    unigrams = []

    with open(arpa_file, "r") as file:
        start_extraction = False
        for line in file:
            line = line.strip()
            if start_extraction:
                if not line:  
                    break
                parts = line.split("\t")
                unigram = parts[1]  
                unigrams.append(unigram)
            elif line.startswith("\\1-grams:"):
                start_extraction = True

    with open(output_file, "w") as file:
        for unigram in unigrams:
            file.write(unigram + "\n")